In [1]:
from pharmbio.dataset import get_projects_list, get_qc_data, get_qc_info
from pharmbio import dataset

get_projects_list(lookup="cov")

['Covid19-Profiling', 'sarscov2-repurposing']

In [2]:
# pharmbio equivalant
from pharmbio.dataset import ExperimentData

sarscov2 = ExperimentData(name="sarscov2-repurposing", filter={'plate_barcode': ['PE']})

Quering the db for sarscov2-repurposing found 1 study: ['sarscov2-repurposing']
__________________________________________________
1
	sarscov2-repurposing
	['001_A', '001_B', '002_A', '002_B', '003_A', '003_B', '004_A', '004_B', '005_A', '005_B', '006_A', '006_B', '007_A', '007_B', '008_A', '008_B', '009_A', '009_B', '010_A', '010_B', '011_A', '011_B', '012_A', '012_B', '013_A', '013_B', '014_A', '014_B', '015_A', '015_B', '016_A', '016_B', 'P102337-VeroE6', 'P102338-A549', 'P102339-VeroE6', 'P102340-A549', 'P102341-VeroE6', 'P102342-A549', 'P102398-VeroE6', 'P102399-A549', 'PE000049-veroE6-sarscov2', 'PE000050-veroE6-sarscov2', 'PE000051-veroE6-sarscov2', 'PE000052-veroE6-sarscov2', 'PE000053-veroE6', 'SS-QC1']
__________________________________________________
No replicated analysis has been found!


Successfully imported (1536, 588): /share/data/cellprofiler/automation/results/PE000049-veroE6-sarscov2/1005/348/qcRAW_images_PE000049-veroE6-sarscov2.csv
Successfully imported (1536, 58

In [3]:
from pharmbio.qc import get_qc_module

get_qc_module(sarscov2.qc_data)

['Correlation',
 'FocusScore',
 'LocalFocusScore',
 'MADIntensity',
 'MaxIntensity',
 'MeanIntensity',
 'MedianIntensity',
 'MinIntensity',
 'PercentMaximal',
 'PercentMinimal',
 'PowerLogLogSlope',
 'Scaling',
 'StdIntensity',
 'ThresholdBackground',
 'ThresholdKapur',
 'ThresholdMCT',
 'ThresholdMoG',
 'ThresholdOtsu',
 'ThresholdRidlerCalvard',
 'ThresholdRobustBackground',
 'TotalArea',
 'TotalIntensity']

In [4]:
from pharmbio.qc import get_qc_data_dict
import re

get_qc_data_dict(sarscov2.qc_data, module_to_keep={'Correlation'})['Correlation']

[
    re.sub('^.*?_.*?_', '', c)
    for c in list(get_qc_data_dict(sarscov2.qc_data, module_to_keep={'PowerLogLogSlope'})['PowerLogLogSlope'].columns)
]

['CONC', 'HOECHST', 'MITO', 'PHAandWGA', 'SYTO']

In [5]:
from pharmbio.qc import get_channels

get_channels(sarscov2.qc_data)

def pretty_print_dict(d):
    for module, data in d.items():
        print(module)
        print("  Channels:", data['channels'])
        print("  Sub-channels:", data['sub_channels'])
        print()
        
pretty_print_dict(get_channels(sarscov2.qc_data, qc_module_list=['Correlation', 'FocusScore']))


Correlation
  Channels: ['CONC', 'HOECHST', 'MITO', 'PHAandWGA', 'SYTO']
  Sub-channels: ['CONC_10', 'CONC_30', 'HOECHST_200', 'HOECHST_80', 'MITO_30', 'MITO_5', 'PHAandWGA_20', 'PHAandWGA_200', 'PHAandWGA_80', 'SYTO_20', 'SYTO_5']

FocusScore
  Channels: ['CONC', 'HOECHST', 'MITO', 'PHAandWGA', 'SYTO']
  Sub-channels: []



In [18]:
from pharmbio.qc import flag_outliers

print(flag_outliers(sarscov2.qc_data, method='IQR', IQR_normalization=True, quantile_limit=0.2).select(["outlier_flag"]).sum())
flag_outliers(sarscov2.qc_data, method='IQR', IQR_normalization=True, quantile_limit=0.2)

shape: (1, 1)
┌──────────────┐
│ outlier_flag │
│ ---          │
│ i64          │
╞══════════════╡
│ 356          │
└──────────────┘


AreaOccupied_AreaOccupied_nuclei,AreaOccupied_Perimeter_nuclei,AreaOccupied_TotalArea_nuclei,Count_nuclei,ExecutionTime_01LoadData,ExecutionTime_02MeasureImageQuality,ExecutionTime_03FlagImage,ExecutionTime_04MeasureImageQuality,ExecutionTime_05FlagImage,ExecutionTime_06MeasureImageQuality,ExecutionTime_07FlagImage,ExecutionTime_08MeasureImageQuality,ExecutionTime_09FlagImage,ExecutionTime_10MeasureImageQuality,ExecutionTime_11FlagImage,ExecutionTime_12IdentifyPrimaryObjects,ExecutionTime_13MeasureImageAreaOccupied,ExecutionTime_14MeasureObjectSizeShape,ExecutionTime_15CalculateMath,ExecutionTime_16CalculateMath,FileName_CONC,FileName_HOECHST,FileName_MITO,FileName_PHAandWGA,FileName_SYTO,Group_Index,Group_Length,Group_Number,Height_CONC,Height_HOECHST,Height_MITO,Height_PHAandWGA,Height_SYTO,ImageNumber,ImageQuality_Correlation_CONC_10,ImageQuality_Correlation_CONC_30,ImageQuality_Correlation_HOECHST_200,…,StDev_nuclei_AreaShape_Zernike_9_9,StDev_nuclei_Location_Center_X,StDev_nuclei_Location_Center_Y,StDev_nuclei_Location_Center_Z,Threshold_FinalThreshold_nuclei,Threshold_GuideThreshold_nuclei,Threshold_OrigThreshold_nuclei,Threshold_SumOfEntropies_nuclei,Threshold_WeightedVariance_nuclei,URL_CONC,URL_HOECHST,URL_MITO,URL_PHAandWGA,URL_SYTO,Width_CONC,Width_HOECHST,Width_MITO,Width_PHAandWGA,Width_SYTO,qc_flag_rawCONC_Blurred,qc_flag_rawCONC_Saturated,qc_flag_rawHOECHST_Blurry,qc_flag_rawHOECHST_Saturated,qc_flag_rawMITO_Blurry,qc_flag_rawMITO_Saturated,qc_flag_rawPHAandWGA_Blurry,qc_flag_rawPHAandWGA_Saturated,qc_flag_rawSYTO_Blurred,qc_flag_rawSYTO_Saturated,Metadata_AcqID,ImageID,OutlierIQR_FocusScore_-3.586_3.527,OutlierIQR_MaxIntensity_-4.733_5.164,OutlierIQR_MeanIntensity_-3.3_3.348,OutlierIQR_PowerLogLogSlope_-3.503_3.551,OutlierIQR_StdIntensity_-3.51_3.297,outlier_flag
f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,str,str,str,str,str,i32,i32,i32,i32,i32,i32,i32,i32,i32,f32,f32,f32,…,f32,f32,f32,f32,f32,f32,f32,f32,f32,str,str,str,str,str,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,str,i64,i64,i64,i64,i64,i64
343016.0,30603.0,4.6656e6,184.0,4.19,11.3,0.01,13.3,0.0,14.39,0.0,12.69,0.0,13.91,0.0,5.79,0.06,5.46,0.0,0.0,"""PE000049-veroE…","""PE000049-veroE…","""PE000049-veroE…","""PE000049-veroE…","""PE000049-veroE…",0,1,1,2160,2160,2160,2160,2160,1,0.759817,0.475376,0.007568,…,0.001374,600.412659,557.625244,0.0,0.012593,0.016412,0.006421,-12.487138,0.504155,"""file:/share/mi…","""file:/share/mi…","""file:/share/mi…","""file:/share/mi…","""file:/share/mi…",2160,2160,2160,2160,2160,0,0,0,0,0,0,0,0,1,0,1005,"""1005_A01_1""",0,0,0,0,0,0
295356.0,25824.0,4.6656e6,156.0,4.61,14.29,0.01,11.6,0.0,14.43,0.0,14.5,0.0,15.12,0.0,6.61,0.06,5.53,0.0,0.0,"""PE000049-veroE…","""PE000049-veroE…","""PE000049-veroE…","""PE000049-veroE…","""PE000049-veroE…",0,1,1,2160,2160,2160,2160,2160,1,0.798159,0.514687,0.010176,…,0.001422,621.378296,651.543518,0.0,0.012738,0.016715,0.006062,-12.509486,0.516467,"""file:/share/mi…","""file:/share/mi…","""file:/share/mi…","""file:/share/mi…","""file:/share/mi…",2160,2160,2160,2160,2160,0,0,0,0,0,0,0,0,1,0,1005,"""1005_A01_2""",0,0,0,0,0,0
256101.0,22426.0,4.6656e6,135.0,4.12,13.39,0.01,10.7,0.0,14.22,0.0,14.08,0.0,14.21,0.0,5.99,0.06,6.38,0.01,0.0,"""PE000049-veroE…","""PE000049-veroE…","""PE000049-veroE…","""PE000049-veroE…","""PE000049-veroE…",0,1,1,2160,2160,2160,2160,2160,1,0.769253,0.507993,-0.023378,…,0.001072,588.080872,587.593689,0.0,0.012418,0.016532,0.005371,-12.534272,0.537314,"""file:/share/mi…","""file:/share/mi…","""file:/share/mi…","""file:/share/mi…","""file:/share/mi…",2160,2160,2160,2160,2160,0,0,1,0,0,0,0,0,1,0,1005,"""1005_A01_3""",0,0,0,0,0,0
256549.0,23495.0,4.6656e6,148.0,3.7,10.95,0.0,12.97,0.01,13.48,0.0,12.93,0.0,15.25,0.01,7.32,0.06,5.2,0.0,0.0,"""PE000049-veroE…","""PE000049-veroE…","""PE000049-veroE…","""PE000049-veroE…","""PE000049-veroE…",0,1,1,2160,2160,2160,2160,2160,1,0.795237,0.5004,-0.009082,…,0.001493,590.243164,61